In [1]:
##THE HYBRID WITH SLIME AND IALS RESULTS ARE BETTER THAN ANY OTHER ALGORITHM TESTED BEFORE
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices, create_submission, write_submission
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDItemRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeThreeModelsByScores, MergeTwoModelsByScores

In [2]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()
URM_stacked = combine_matrices(ICM=ICM, URM=URM)

In [3]:
##SLIMELASTICNET Recommender
SLIME_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_stacked)
SLIME_recommender.fit(topK=420, l1_ratio=0.06061216785568925, alpha=0.00554982938879437, workers = 10)

100%|████████████████████████████████████▉| 18032/18059 [12:41<00:00, 53.93it/s]

In [4]:
## IALS Recommender
IALS_recommender = IALSRecommender(URM_train=URM_stacked)
IALS_recommender.fit(epochs=10, num_factors=42, alpha=0.7617528864750021, reg=8.926401306541349)

IALSRecommenderLinear: Epoch 1 of 10. Elapsed time 13.70 sec


100%|████████████████████████████████████▉| 18056/18059 [13:00<00:00, 53.93it/s]

IALSRecommenderLinear: Epoch 2 of 10. Elapsed time 34.66 sec
IALSRecommenderLinear: Epoch 3 of 10. Elapsed time 54.45 sec
IALSRecommenderLinear: Epoch 4 of 10. Elapsed time 1.16 min
IALSRecommenderLinear: Epoch 5 of 10. Elapsed time 1.40 min
IALSRecommenderLinear: Epoch 6 of 10. Elapsed time 1.67 min
IALSRecommenderLinear: Epoch 7 of 10. Elapsed time 1.93 min
IALSRecommenderLinear: Epoch 8 of 10. Elapsed time 2.19 min
IALSRecommenderLinear: Epoch 9 of 10. Elapsed time 2.49 min
IALSRecommenderLinear: Epoch 10 of 10. Elapsed time 2.77 min
IALSRecommenderLinear: Terminating at epoch 10. Elapsed time 2.77 min


In [9]:
##ITEMCFCBF
ITEMCFCBF=ItemKNN_CFCBF_Hybrid_Recommender(ICM_train=ICM,URM_train=URM)
ITEMCFCBF.fit(topK=174, shrink=54.93865189896452, similarity='cosine')

ItemKNN_CFCBF_HybridRecommender: ICM Detected 1 ( 0.0%) items with no features.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 18059 (100.0%), 950.16 column/sec. Elapsed time 19.01 sec


In [10]:
ITEMCFCBF_SLIM = MergeTwoModelsByScores(URM_train = URM_stacked, recommenders=[ITEMCFCBF,SLIME_recommender])
ITEMCFCBF_SLIM.fit(alpha=0.006419859589518687,beta=1.0)

In [11]:
ITEMCFCBF_SLIM_IALS = MergeTwoModelsByScores(URM_train = URM_stacked, recommenders=[ITEMCFCBF_SLIM,IALS_recommender])
ITEMCFCBF_SLIM_IALS.fit(alpha=0.7751267998183894,beta=0.35391338107827336)

In [12]:
submission = create_submission(ITEMCFCBF_SLIM_IALS)
write_submission(submission,"ITEMCFCBF_SLIM_IALS")

100%|██████████████████████████████████▉| 18056/18059 [1:09:28<00:00,  4.33it/s]


In [14]:
##CREATION OF DIFFERENT RECOMMENDERS TUNED

### RP3Beta Recommender
RP3Beta_recommender = RP3betaRecommender(URM_train=URM)
RP3Beta_recommender.fit(topK=181, alpha=0.5709402717259106, beta=0.3566066452521787, implicit=True, normalize_similarity=True)

In [20]:
##RP3HYBRIDSLIME_IALS
similarity_slime = SLIME_recommender.W_sparse
similarity_rp3 = RP3Beta_recommender.W_sparse

similarity_hybrid = ItemKNNSimilarityHybridRecommender(URM_train = URM_stacked,
                                                      Similarity_1 = similarity_rp3,
                                                      Similarity_2 = similarity_slime)
similarity_hybrid.fit(topK=494, alpha=0.1597863887003789 )
recommenders =[similarity_hybrid,IALS_recommender]
hybrid = MergeTwoModelsByScores(URM_stacked, recommenders)
hybrid.fit(alpha=0.4555360263375717,beta=0.13654581079541908)

In [21]:
ITEMCFCBF_SLIM_HYBRID =MergeTwoModelsByScores(URM_train = URM_stacked, recommenders=[ITEMCFCBF_SLIM_IALS,hybrid])
ITEMCFCBF_SLIM_HYBRID.fit(alpha=0.5134425691700654,beta=0.44336807279575097)

In [22]:
submission = create_submission(ITEMCFCBF_SLIM_HYBRID)
write_submission(submission,"ITEMCFCBF_SLIM_HYBRID")